In [2]:
from dotenv import load_dotenv
from pdf2image import pdfinfo_from_path, convert_from_path
import os

import json
import base64
from pathlib import Path

import pandas as pd
from PIL import Image
from openai import OpenAI

POPPLER_PATH = r"C:\poppler\Library\bin" #Estoy usando el equipo de trabajo con Windows, tuve que configurar esta línea por que me estaba generando conflictos.
PDF_PATH = "SMAE-5a-ed-Ana-Bertha-Perez-Lizaur.pdf" 
PAGES_DIR = "pages"       
OUTPUT_CSV = "smae_tablas.csv"

# Api key de OPENAI_API_KEY

load_dotenv()

client = OpenAI(api_key=os.getenv("OPENAI_API_KEY"))
os.makedirs(PAGES_DIR, exist_ok=True)


In [3]:
info = pdfinfo_from_path(PDF_PATH, poppler_path=POPPLER_PATH)
num_pages = info["Pages"]

print("Total de páginas:", num_pages)

for page_num in range(1, num_pages + 1):
    print(f"Convirtiendo página {page_num}...")
    images = convert_from_path(
        PDF_PATH,
        dpi=180,             
        first_page=page_num,
        last_page=page_num,
        poppler_path=POPPLER_PATH
    )
    img = images[0]
    img.save(f"{PAGES_DIR}/page_{page_num:03d}.png", "PNG")

print("Conversión completada sin saturar memoria.")


Total de páginas: 208
Convirtiendo página 1...
Convirtiendo página 2...
Convirtiendo página 3...
Convirtiendo página 4...
Convirtiendo página 5...
Convirtiendo página 6...
Convirtiendo página 7...
Convirtiendo página 8...
Convirtiendo página 9...
Convirtiendo página 10...
Convirtiendo página 11...
Convirtiendo página 12...
Convirtiendo página 13...
Convirtiendo página 14...
Convirtiendo página 15...
Convirtiendo página 16...
Convirtiendo página 17...
Convirtiendo página 18...
Convirtiendo página 19...
Convirtiendo página 20...
Convirtiendo página 21...
Convirtiendo página 22...
Convirtiendo página 23...
Convirtiendo página 24...
Convirtiendo página 25...
Convirtiendo página 26...
Convirtiendo página 27...
Convirtiendo página 28...
Convirtiendo página 29...
Convirtiendo página 30...
Convirtiendo página 31...
Convirtiendo página 32...
Convirtiendo página 33...
Convirtiendo página 34...
Convirtiendo página 35...
Convirtiendo página 36...
Convirtiendo página 37...
Convirtiendo página 38...

In [4]:
def encode_image_to_base64(path):
    with open(path, "rb") as f:
        return base64.b64encode(f.read()).decode("utf-8")

def extraer_tabla_smae(image_path):
    """
    Envía una página a GPT y devuelve JSON válido gracias a response_format=json_object.
    """
    b64 = encode_image_to_base64(image_path)

    prompt = """
Eres un experto en lectura de tablas del SMAE.
Extrae la tabla principal de la imagen y devuélvela en este formato EXACTO:

{
  "categoria": "texto o null",
  "alimentos": [
     {
       "alimento": "",
       "unidad": "",
       "peso_crudo_g": null,
       "peso_neto_g": null,
       "energia_kcal": null,
       "proteina_g": null,
       "lipidos_g": null,
       "hidratos_carbono_g": null,
       "fibra_g": null,
       "azucar_g": null,
       "colesterol_mg": null,
       "acidos_saturados_g": null,
       "acidos_monoinsaturados_g": null,
       "acidos_poliinsaturados_g": null,
       "acido_folico_mcg": null,
       "vitamina_a_mcg": null,
       "calcio_mg": null,
       "hierro_mg": null,
       "sodio_mg": null,
       "potasio_mg": null,
       "fosforo_mg": null,
       "zinc_mg": null,
       "selenio_mcg": null,
       "carga_glicemica": null
    }
  ]
}

Reglas IMPORTANTES:
- Usa SIEMPRE un JSON válido. No añadas texto antes ni después.
- Si alguna casilla aparece como "ND" o está vacía, pon el valor como null.
- Si algún nutriente no aparece en la tabla de esa página, llena todos sus valores con null.
- No inventes filas que no estén en la tabla.
"""
    response = client.chat.completions.create(
        model="gpt-4.1-mini",
        response_format={"type": "json_object"},
        messages=[
            {
                "role": "user",
                "content": [
                    {"type": "text", "text": prompt},
                    {"type": "image_url",
                     "image_url": {
                         "url": f"data:image/png;base64,{b64}"
                     }}
                ]
            }
        ]
    )

    json_data = response.choices[0].message.content
    return json.loads(json_data)


In [5]:
all_rows = []

page_files = sorted(os.listdir(PAGES_DIR))

print("Procesando", len(page_files), "páginas…")

for fname in page_files:
    image_path = os.path.join(PAGES_DIR, fname)
    print("Procesando:", fname)
    
    try:
        data = extraer_tabla_smae(image_path)
        categoria = data.get("categoria")
        
        for fila in data.get("alimentos", []):
            fila["categoria"] = categoria
            fila["pagina"] = fname
            all_rows.append(fila)

    except Exception as e:
        print("❌ Error en", fname, "→", e)




Procesando 208 páginas…
Procesando: page_001.png
Procesando: page_002.png
Procesando: page_003.png
Procesando: page_004.png
Procesando: page_005.png
Procesando: page_006.png
Procesando: page_007.png
Procesando: page_008.png
Procesando: page_009.png
Procesando: page_010.png
Procesando: page_011.png
Procesando: page_012.png
Procesando: page_013.png
Procesando: page_014.png
Procesando: page_015.png
Procesando: page_016.png
Procesando: page_017.png
Procesando: page_018.png
Procesando: page_019.png
Procesando: page_020.png
Procesando: page_021.png
Procesando: page_022.png
Procesando: page_023.png
Procesando: page_024.png
Procesando: page_025.png
Procesando: page_026.png
Procesando: page_027.png
Procesando: page_028.png
Procesando: page_029.png
Procesando: page_030.png
Procesando: page_031.png
Procesando: page_032.png
Procesando: page_033.png
Procesando: page_034.png
Procesando: page_035.png
Procesando: page_036.png
Procesando: page_037.png
Procesando: page_038.png
Procesando: page_039.png
P

In [6]:
df = pd.DataFrame(all_rows)
print("Total de filas:", len(df))
df.head()

Total de filas: 2761


,alimento,unidad,peso_crudo_g,peso_neto_g,energia_kcal,proteina_g,lipidos_g,hidratos_carbono_g,fibra_g,azucar_g,...,AG saturados_g,AG monoinsaturados_g,AG poliinsaturados_g,cantidad_sugerida,peso_bruto_g,ac_g_saturados_g,ac_monoinsaturados_g,ac_poliinsaturados_g,AG poliinsaturados (g),AG monoinsaturados (g)
0,verduras,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,frutas,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,cereales sin grasa,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,cereales con grasa,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,leguminosas,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [7]:
df.to_csv(OUTPUT_CSV, index=False, encoding="utf-8-sig")
print("CSV guardado como:", OUTPUT_CSV)


✔ CSV guardado como: smae_tablas.csv
